In [1]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm,tqdm_notebook
parent_path='../Data/New_Data_15-06-2020/'

In [2]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
whatsapp_data=pd.read_csv(parent_path+'Data_text_spam_removed_v02_hi_en.csv')
temp=whatsapp_data[['group_id_anonymized','phone_num_anonymized','message_text','timestamp']]
duplicateDFRow = temp[temp.duplicated()]
whatsapp_data=whatsapp_data.drop(list(duplicateDFRow.index))

In [ ]:
from langdetect import detect
from langdetect import DetectorFactory
import parmap

DetectorFactory.seed = 0

def add_language(df):
    language= []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            lang=detect(row["message_text"])
            language.append(lang)
        except:
            language.append('unk')

    df["language"] = language
    return df

n_process=30


df_split = np.array_split(whatsapp_data, n_process)
list_all = parmap.map(add_language, df_split,pm_processes=n_process,pm_pbar=True)
whatsapp_data=pd.concat(list_all,axis=0)

In [ ]:
whatsapp_data=whatsapp_data[whatsapp_data['language'].isin(['hi','en'])]


In [3]:
import numpy as np

def Match(df, caliper = 0.05, caliper_method = "propensity", replace = False):
        
    
    # Transform the propensity scores and caliper when caliper_method is "logit" or "none"
    if caliper_method == "logit":
        propensity = log(propensity/(1-propensity))
        caliper = caliper*np.std(propensity)
    elif caliper_method == "none":
        caliper = 0
    
    # Code groups as 0 and 1
    #groups = groups == np.unique(groups)
    N = len(df)
    N1 = df[df['treatment'] == 1].index; N2 =df[df['treatment'] == 0].index
    g1, g2 = df.loc[N1]['pscore'],df.loc[N2]['pscore']
    # Check if treatment groups got flipped - the smaller should correspond to N1/g1
    if len(N1) > len(N2):
       N1, N2, g1, g2 = N2, N1, g2, g1
        
        
    # Randomly permute the smaller group to get order for matching
    morder = np.random.permutation(N1)
    matches = {}

    
    for m in morder:
        dist = abs(g1[m] - g2)
        if (dist.min() <= caliper) or not caliper:
            matches[m] = dist.idxmin()    # Potential problem: check for ties
            if not replace:
                g2 = g2.drop(matches[m])
    return (matches)

def whichMatched(matches, data, many = False, unique = False):
    ''' 
    Simple function to convert output of Matches to DataFrame of all matched observations
    Inputs:
    matches = output of Match
    data = DataFrame of covariates
    many = Boolean indicating if matching method is one-to-one or one-to-many
    unique = Boolean indicating if duplicated individuals (ie controls matched to more than one case) should be removed
    '''

    tr = matches.keys()
    if many:
        ctrl = [m for matchset in matches.values() for m in matchset]
    else:
        ctrl = matches.values()
    # need to remove duplicate rows, which may occur in matching with replacement
    temp = pd.concat([data.loc[tr], data.loc[ctrl]])
    if unique == True:
        return temp.groupby(temp.index).first()
    else:
        return temp


In [4]:
from datetime import datetime


def convert(year,month,date):
    return int(datetime(year, month, date, 0, 0, 0).timestamp()*1000)

def convert_reverse(timestamp):
    dt_object = datetime.fromtimestamp(timestamp/1000)
    print("dt_object =", dt_object)
    return dt_object

In [ ]:
convert_reverse(1530948979856)

In [ ]:
whatsapp_data=whatsapp_data.sort_values('timestamp')

In [ ]:
whatsapp_data.head(5)

In [5]:
all_fear_speech_index=[]
all_normal_index=[]

count_fearspeech=0
count_normal=0

for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if(row['one_fear_speech']==1):
        count_fearspeech+=1
        all_fear_speech_index+=row['repeated']
    elif(row['one_fear_speech']==0):
        count_normal+=1
        all_normal_index+=row['repeated']

data_fear_speech=whatsapp_data[whatsapp_data['orig_index'].isin(all_fear_speech_index)]
data_normal=whatsapp_data[whatsapp_data['orig_index'].isin(all_normal_index)]


100%|██████████| 4782/4782 [00:00<00:00, 14913.93it/s]


In [25]:
import numpy as np

tuples_list=[]

for user in tqdm(whatsapp_data.phone_num_anonymized.unique(),total=whatsapp_data.phone_num_anonymized.nunique()):
    temp={}
    temp['user_id']=user
    user_data=whatsapp_data[whatsapp_data['phone_num_anonymized']==user]
    if(user in set(data_fear_speech.phone_num_anonymized.unique())):
        temp['treatment']=1
    else:
        temp['treatment']=0
        
   
    temp['num_groups']=user_data['group_id_anonymized'].nunique()
    count=0
    list_message_present=[]
    for year in [2018,2019]:
        for month in range(1,13):
            timestamp=convert(year,month,1)
            count+=1
            temp_data=user_data[user_data['timestamp'].between(timestamp, timestamp+30*24*60*60*1000, inclusive=True)]
            if(len(temp_data)>0):
                try:
                    start=temp['start_date']
                except KeyError:
                    temp['start_date']=count
                list_message_present.append(len(temp_data))
    if(len(list_message_present)>0):
        temp['mean_messages']=np.mean(list_message_present)
        temp['stdev_messages']=np.std(list_message_present)
        temp['month_present']=len(list_message_present)
        tuples_list.append(temp)         


  0%|          | 303/109542 [00:03<21:49, 83.41it/s]


KeyboardInterrupt: 

In [7]:
df = pd.DataFrame(tuples_list) 

In [8]:
df

,user_id,treatment,num_groups,start_date,mean_messages,stdev_messages,month_present
0,2148,0,154,7,24.571429,21.181914,14
1,247483,0,1,7,10.000000,1.632993,3
2,1287,1,1,7,4.333333,3.399346,3
3,244632,0,2,7,5.000000,3.674235,4
4,242857,0,1,7,32.333333,24.115463,3
...,...,...,...,...,...,...,...
108692,23730,0,1,20,1.000000,0.000000,1
108693,174891,0,1,20,1.000000,0.000000,1
108694,174892,0,1,20,20.000000,0.000000,1
108695,174080,0,1,20,1.000000,0.000000,1


In [14]:
from pscore_match.pscore import PropensityScore
#from pscore_match.match import Match, whichMatched
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


treatment = np.array(df.treatment)
cov_list = ['num_groups','start_date','mean_messages','stdev_messages','month_present']
#cov_list = ['start_date','month_present']

covariates = df[cov_list]

pscore = PropensityScore(treatment, covariates).compute()
df['pscore']=pscore
pairs = Match(df,caliper = 0.05)
data_matched=whichMatched(pairs, df,many = False, unique = False)
# pairs = Match(treatment, pscore)
# pairs.create(method='one-to-one',caliper=0.001,replace=False)
# data_matched = whichMatched(pairs, pd.DataFrame({'pscore': pscore, 'treatment' :treatment, 'user_id':df.user_id}))


In [15]:
treatment=np.array(data_matched[data_matched['treatment']==1].pscore)
control=np.array(data_matched[data_matched['treatment']==0].pscore)

In [16]:
data_fear_speech.phone_num_anonymized.nunique()

2933

In [17]:
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu
# seed the random number generator
seed(1)
# generate two independent samples
# compare samples
stat, p = mannwhitneyu(treatment, control)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')

Statistics=4298233.000, p=0.500
Same distribution (fail to reject H0)


In [18]:
len(data_matched)/2

2932.0

In [20]:
matched_groups=[]
for index,row in data_matched.iterrows():
    if(row['treatment']==1):
        matched_groups.append([int(row['user_id']),'Fear speech'])
    else:
        matched_groups.append([int(row['user_id']),'Non fear speech'])
    

In [26]:
data_matched

,user_id,treatment,num_groups,start_date,mean_messages,stdev_messages,month_present,pscore
93971,219636,1,1,17,2.000,0.816497,3,0.048243
5874,237650,1,1,9,10.000,4.949747,4,0.031172
82294,177266,1,1,16,4.000,1.632993,3,0.043026
557,26029,1,1,7,7.375,4.820205,8,0.184425
84460,179003,1,1,16,2.250,1.639360,4,0.072255
...,...,...,...,...,...,...,...,...
94057,144207,0,1,17,4.500,3.500000,2,0.028946
25930,67132,0,1,13,1.750,0.829156,4,0.049133
13588,151715,0,1,11,6.625,2.735759,8,0.276065
10535,94013,0,3,10,2.875,2.204399,8,0.225246


In [27]:
df_group_matched=pd.DataFrame(matched_groups,columns=['user_id','annotation'])

In [28]:
df_group_matched.to_csv(parent_path+'final_user_selection_matched.csv',index=False)

In [23]:
df_group_matched

,user_id,annotation
0,219636,Fear speech
1,237650,Fear speech
2,177266,Fear speech
3,26029,Fear speech
4,179003,Fear speech
...,...,...
5859,144207,Non fear speech
5860,67132,Non fear speech
5861,151715,Non fear speech
5862,94013,Non fear speech


### Group matching


In [ ]:
from collections import OrderedDict 
group_dict={}
for group in data_fear_speech.group_id_anonymized.unique():
    temp=data_fear_speech[data_fear_speech['group_id_anonymized']==group]
    group_dict[group]=len(temp)
dict1 = sorted(group_dict.items(),key=lambda item: item[1],reverse=True)

selected_groups=[ele[0] for ele in dict1[0:100]]
len(selected_groups)

In [ ]:
import numpy as np

tuples_list_grp=[]

for user in tqdm_notebook(whatsapp_data.group_id_anonymized.unique(),total=whatsapp_data.group_id_anonymized.nunique()):
    temp={}
    temp['group_id']=user
    user_data=whatsapp_data[whatsapp_data['group_id_anonymized']==user]
    if(user in data_fear_speech.group_id_anonymized.unique()):
        if(user in selected_groups):
            temp['treatment']=1
        else:
            temp['treatment']=-1
    else:
        temp['treatment']=0
        
   
    temp['num_users']=user_data['phone_num_anonymized'].nunique()
    count=0
    list_message_present=[]
    for year in [2018,2019]:
        for month in range(1,13):
            timestamp=convert(year,month,1)
            count+=1
            temp_data=user_data[user_data['timestamp'].between(timestamp, timestamp+30*24*60*60*1000, inclusive=True)]
            if(len(temp_data)>0):
                try:
                    start=temp['start_date']
                except KeyError:
                    temp['start_date']=count
                list_message_present.append(len(temp_data))
    if(len(list_message_present)>0):
        temp['mean_messages']= len(user_data)/temp['num_users']
        temp['stdev_messages']=np.std(list_message_present)
        temp['month_present']=len(list_message_present)
        tuples_list_grp.append(temp)         



In [ ]:
df_groups = pd.DataFrame(tuples_list_grp) 

In [ ]:
df_groups = df_groups[df_groups['treatment']!=-1]

In [ ]:
len(df_groups[df_groups['treatment']==1])

In [ ]:
from pscore_match.pscore import PropensityScore
#from pscore_match.match import Match, whichMatched
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


treatment = np.array(df_groups.treatment)
cov_list = ['start_date','num_users','month_present','mean_messages']
#cov_list = ['start_date','month_present']

covariates = df_groups[cov_list]

pscore = PropensityScore(treatment, covariates).compute()
df_groups['pscore']=pscore
pairs = Match(df_groups,caliper = 0.1)
data_matched=whichMatched(pairs, df_groups, many = False, unique = False)
# pairs = Match(treatment, pscore)
# pairs.create(method='one-to-one',caliper=0.001,replace=False)
# data_matched = whichMatched(pairs, pd.DataFrame({'pscore': pscore, 'treatment' :treatment, 'user_id':df.user_id}))


In [ ]:
treatment=np.array(data_matched[data_matched['treatment']==1].pscore)
control=np.array(data_matched[data_matched['treatment']==0].pscore)


#print(data_fear_speech.group_id_anonymized.nunique())
print(len(treatment))

In [ ]:
t

In [ ]:
from scipy.stats import pearsonr
# seed random number generator
seed(1)
# prepare data
data1 = 20 * randn(1000) + 100
data2 = data1 + (10 * randn(1000) + 50)
# calculate Pearson's correlation
corr, _ = pearsonr(df_groups['start_date'], df_groups['month_present'])
print('Pearsons correlation: %.3f' % corr)

In [ ]:
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu
# seed the random number generator
seed(2)
# generate two independent samples
# compare samples
stat, p = mannwhitneyu(treatment, control)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.005
if p > alpha:
	print('Same distribution (fail to reject H0)')
else:
	print('Different distribution (reject H0)')

In [ ]:
fear_speech_groups=data_matched[data_matched['treatment']==1]['group_id']
non_fear_speech_groups=data_matched[data_matched['treatment']==0]['group_id']


fear_speech_groups_user=list(whatsapp_data[whatsapp_data['group_id_anonymized'].isin(fear_speech_groups)].phone_num_anonymized.unique())
non_fear_speech_groups_user=list(whatsapp_data[whatsapp_data['group_id_anonymized'].isin(non_fear_speech_groups)].phone_num_anonymized.unique())

In [ ]:
common_users=set(non_fear_speech_groups_user).intersection(set(fear_speech_groups_user))
fear_speech_groups_user=set(fear_speech_groups_user).difference(common_users)
non_fear_speech_groups_user=set(non_fear_speech_groups_user).difference(common_users)

fear_speech_groups_user=fear_speech_groups_user.difference(set(data_fear_speech.phone_num_anonymized.unique()))
non_fear_speech_groups_user=non_fear_speech_groups_user.difference(set(data_fear_speech.phone_num_anonymized.unique()))


In [ ]:
print(len(fear_speech_groups_user),len(non_fear_speech_groups_user),len(data_fear_speech.phone_num_anonymized.unique()))



tuple_list=[]

for ele in fear_speech_groups_user:
    tuple_list.append([ele,'in_FS_group'])
for ele in non_fear_speech_groups_user:
    tuple_list.append([ele,'in_NFS_group'])
for ele in data_fear_speech.phone_num_anonymized.unique():
    tuple_list.append([ele,'FS_user'])
    
    

In [ ]:
df_users_survey=pd.DataFrame(tuple_list,columns=['user_id','label'])

In [ ]:
df_users_survey.to_csv('users_for_survey.csv',index=False)

In [ ]:
df_users_survey.label.unique()

In [ ]:
whatsapp_data.phone_num_anonymized.unique()

In [ ]:
matched_groups=[]
for index,row in data_matched.iterrows():
    if(row['treatment']==1):
        matched_groups.append([int(row['group_id']),'Fear speech'])
    else:
        matched_groups.append([int(row['group_id']),'Non fear speech'])
    

In [ ]:
df_group_matched=pd.DataFrame(matched_groups,columns=['grp_id','annotation'])

In [ ]:
df_group_matched.to_csv(parent_path+'final_group_selection_matched.csv',index=False)

In [ ]:
df_group_matched

In [ ]:
len(treatment)